# Bluesky API Tutorial for Political Science Research 🗳️📊

## Introduction to Social Media Data Collection Using Bluesky's AT Protocol

**Target Audience:** Political Science Graduate Students  
**Prerequisites:** Basic familiarity with spreadsheets; no Python experience required  
**Platform:** Google Colab (runs in your web browser)  
**Duration:** 60-90 minutes

---

## What You'll Learn 🎯

By the end of this tutorial, you'll be able to:
1. **Collect social media data** from Bluesky for political research
2. **Analyze user networks** and political discourse patterns
3. **Export data** for use in statistical software (R, SPSS, Stata)
4. **Visualize political conversations** and community structures
5. **Understand ethical considerations** in social media research

---

## Part 1: Getting Started with Google Colab

### Step 1: Setting Up Your Environment

```python
# First, let's install the required libraries
# The exclamation mark (!) tells Colab to run this as a system command
!pip install atproto pandas matplotlib seaborn networkx wordcloud

# Import the libraries we'll use
import pandas as pd  # For data analysis (like Excel but more powerful)
import matplotlib.pyplot as plt  # For creating charts
import seaborn as sns  # For prettier charts
import networkx as nx  # For network analysis
from wordcloud import WordCloud  # For word clouds
import json  # For handling data
from datetime import datetime, timedelta
import time  # For adding delays between API calls

# Configure display settings
plt.style.use('default')
sns.set_palette("husl")

print("✅ All libraries installed successfully!")
print("📋 Ready to begin data collection from Bluesky")
```

### Step 2: Understanding Bluesky and the AT Protocol

**What is Bluesky?**
- Bluesky is a decentralized social media platform
- Built on the AT Protocol (Authenticated Transfer Protocol)
- Users have unique identifiers called DIDs (Decentralized Identifiers)
- All public data is accessible for research purposes

**Why is this useful for political science research?**
- Study political discourse in real-time
- Analyze network structures of political communities
- Track information flow during campaigns or crises
- Examine public opinion formation

---

## Part 2: Authentication and Basic Setup

### Step 3: Setting Up Bluesky API Access

```python
# Import the Bluesky SDK
from atproto import Client

# Initialize the client
client = Client()

# For this tutorial, we'll use public data (no authentication required)
# If you need to access private data or post content, you would use:
# client.login('your_handle.bsky.social', 'your_password')

print("🔗 Successfully connected to Bluesky API")
print("📖 Ready to access public data")
```

**Important Note for Researchers:**
- Most political research uses public data, which doesn't require authentication
- If you need to authenticate, create an "App Password" in Bluesky settings (never use your main password)
- Always follow your institution's IRB guidelines for social media research

---

## Part 3: Basic Data Collection

### Step 4: Understanding User Identification

```python
def resolve_handle_to_did(handle):
    """
    Convert a Bluesky handle (like 'user.bsky.social') to a DID
    DIDs are permanent identifiers, handles can change
    """
    try:
        response = client.com.atproto.identity.resolve_handle({'handle': handle})
        return response.did
    except Exception as e:
        print(f"❌ Could not resolve handle {handle}: {e}")
        return None

# Example: Let's look up some political accounts
political_handles = [
    'bsky.app',  # Official Bluesky account
    'atproto.com',  # AT Protocol account
]

print("🔍 Resolving political accounts to DIDs:")
political_dids = {}
for handle in political_handles:
    did = resolve_handle_to_did(handle)
    if did:
        political_dids[handle] = did
        print(f"✅ {handle} → {did}")
    else:
        print(f"❌ Failed to resolve {handle}")
```

### Step 5: Collecting User Posts

```python
def get_user_posts(did, limit=20):
    """
    Collect recent posts from a specific user
    Returns a list of post data
    """
    try:
        response = client.com.atproto.repo.list_records({
            'repo': did,
            'collection': 'app.bsky.feed.post',
            'limit': limit
        })
        
        posts = []
        for record in response.records:
            post_data = {
                'uri': record.uri,
                'text': record.value.text,
                'created_at': record.value.created_at,
                'author_did': did,
                'reply_count': getattr(record.value, 'reply_count', 0),
                'repost_count': getattr(record.value, 'repost_count', 0),
                'like_count': getattr(record.value, 'like_count', 0)
            }
            posts.append(post_data)
        
        return posts
    except Exception as e:
        print(f"❌ Error collecting posts: {e}")
        return []

# Collect posts from our political accounts
print("📝 Collecting recent posts...")
all_posts = []

for handle, did in political_dids.items():
    print(f"Collecting posts from {handle}...")
    user_posts = get_user_posts(did, limit=10)
    all_posts.extend(user_posts)
    time.sleep(1)  # Be respectful to the API

print(f"✅ Collected {len(all_posts)} posts total")
```

### Step 6: Converting to DataFrame for Analysis

```python
# Convert our posts to a pandas DataFrame (like an Excel spreadsheet)
posts_df = pd.DataFrame(all_posts)

if not posts_df.empty:
    # Convert timestamp to readable date
    posts_df['created_at'] = pd.to_datetime(posts_df['created_at'])
    posts_df['date'] = posts_df['created_at'].dt.date
    
    # Add text length column
    posts_df['text_length'] = posts_df['text'].str.len()
    
    print("📊 Data Summary:")
    print(f"Number of posts: {len(posts_df)}")
    print(f"Date range: {posts_df['date'].min()} to {posts_df['date'].max()}")
    print(f"Average post length: {posts_df['text_length'].mean():.1f} characters")
    
    # Display first few posts
    print("\n📖 Sample posts:")
    display(posts_df[['text', 'created_at', 'like_count']].head())
else:
    print("⚠️ No posts collected. Try different accounts or check your connection.")
```

---

## Part 4: Data Analysis for Political Research

### Step 7: Temporal Analysis of Political Activity

```python
# Analyze posting patterns over time
if not posts_df.empty:
    # Group posts by date
    daily_posts = posts_df.groupby('date').size().reset_index(name='post_count')
    
    # Create visualization
    plt.figure(figsize=(12, 6))
    plt.plot(daily_posts['date'], daily_posts['post_count'], marker='o')
    plt.title('Daily Posting Activity', fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Number of Posts')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("📈 Temporal Analysis Complete")
    print("💡 Research Applications:")
    print("   • Track political engagement during events")
    print("   • Identify patterns in campaign communication")
    print("   • Analyze response to political crises")
```

### Step 8: Content Analysis with Word Clouds

```python
# Create word cloud from post content
if not posts_df.empty:
    # Combine all post text
    all_text = ' '.join(posts_df['text'].astype(str))
    
    # Remove common words that aren't meaningful for political analysis
    from wordcloud import STOPWORDS
    political_stopwords = STOPWORDS.union({
        'https', 'http', 'com', 'www', 'co', 'amp', 'rt', 'via'
    })
    
    # Generate word cloud
    wordcloud = WordCloud(
        width=800,
        height=400,
        background_color='white',
        stopwords=political_stopwords,
        max_words=100,
        colormap='viridis'
    ).generate(all_text)
    
    # Display
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Most Common Terms in Political Posts', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("☁️ Word Cloud Analysis Complete")
    print("💡 Research Applications:")
    print("   • Identify key political themes")
    print("   • Track issue salience over time")
    print("   • Compare discourse across different actors")
```

### Step 9: Engagement Analysis

```python
# Analyze engagement patterns
if not posts_df.empty:
    # Calculate engagement metrics
    posts_df['total_engagement'] = (
        posts_df['like_count'] +
        posts_df['repost_count'] +
        posts_df['reply_count']
    )
    
    # Create engagement visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Engagement by post length
    ax1.scatter(posts_df['text_length'], posts_df['total_engagement'], alpha=0.6)
    ax1.set_xlabel('Post Length (characters)')
    ax1.set_ylabel('Total Engagement')
    ax1.set_title('Engagement vs. Post Length')
    ax1.grid(True, alpha=0.3)
    
    # Engagement over time
    daily_engagement = posts_df.groupby('date')['total_engagement'].mean()
    ax2.plot(daily_engagement.index, daily_engagement.values, marker='o')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Average Engagement')
    ax2.set_title('Engagement Trends Over Time')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    # Distribution of engagement types
    engagement_types = ['like_count', 'repost_count', 'reply_count']
    engagement_means = [posts_df[col].mean() for col in engagement_types]
    ax3.bar(engagement_types, engagement_means, color=['#ff7f0e', '#2ca02c', '#d62728'])
    ax3.set_ylabel('Average Count')
    ax3.set_title('Types of Engagement')
    ax3.set_xticklabels(['Likes', 'Reposts', 'Replies'], rotation=45)
    
    # Engagement distribution
    ax4.hist(posts_df['total_engagement'], bins=20, alpha=0.7, color='skyblue')
    ax4.set_xlabel('Total Engagement')
    ax4.set_ylabel('Number of Posts')
    ax4.set_title('Distribution of Post Engagement')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Engagement Analysis Complete")
    print(f"📋 Summary Statistics:")
    print(f"   • Average likes per post: {posts_df['like_count'].mean():.1f}")
    print(f"   • Average reposts per post: {posts_df['repost_count'].mean():.1f}")
    print(f"   • Average replies per post: {posts_df['reply_count'].mean():.1f}")
```

---

## Part 5: Network Analysis for Political Research

### Step 10: Building Follow Networks

```python
def get_user_follows(did, limit=50):
    """
    Get the accounts that a user follows
    Useful for understanding political networks
    """
    try:
        response = client.com.atproto.repo.list_records({
            'repo': did,
            'collection': 'app.bsky.graph.follow',
            'limit': limit
        })
        
        follows = []
        for record in response.records:
            follows.append({
                'follower_did': did,
                'following_did': record.value.subject,
                'created_at': record.value.created_at
            })
        
        return follows
    except Exception as e:
        print(f"❌ Error getting follows: {e}")
        return []

# Build a small network
print("🕸️ Building political follow network...")
network_edges = []

for handle, did in list(political_dids.items())[:2]:  # Limit to first 2 accounts
    print(f"Getting follows for {handle}...")
    follows = get_user_follows(did, limit=10)
    network_edges.extend(follows)
    time.sleep(1)

print(f"✅ Collected {len(network_edges)} network connections")
```

### Step 11: Visualizing Political Networks

```python
if network_edges:
    # Create network graph
    G = nx.DiGraph()  # Directed graph (follows are directional)
    
    # Add edges (connections)
    for edge in network_edges:
        G.add_edge(edge['follower_did'][:10], edge['following_did'][:10])
    
    # Create visualization
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, k=1, iterations=50)
    
    # Draw network
    nx.draw_networkx_nodes(G, pos, node_color='lightblue',
                          node_size=500, alpha=0.7)
    nx.draw_networkx_edges(G, pos, edge_color='gray',
                          arrows=True, arrowsize=20, alpha=0.5)
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    plt.title('Political Follow Network on Bluesky', fontsize=16, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Network statistics
    print("🔍 Network Analysis:")
    print(f"   • Number of nodes (accounts): {G.number_of_nodes()}")
    print(f"   • Number of edges (connections): {G.number_of_edges()}")
    print(f"   • Network density: {nx.density(G):.3f}")
    
    print("\n💡 Research Applications:")
    print("   • Map political communities and coalitions")
    print("   • Identify influential political actors")
    print("   • Track information flow patterns")
    print("   • Study political polarization")
else:
    print("⚠️ No network data available. Try with different accounts.")
```

---

## Part 6: Data Export for Further Analysis

### Step 12: Preparing Data for Statistical Software

```python
# Export data for use in R, SPSS, Stata, etc.
if not posts_df.empty:
    # Clean and prepare data
    export_df = posts_df.copy()
    
    # Create additional variables useful for political analysis
    export_df['hour_posted'] = export_df['created_at'].dt.hour
    export_df['day_of_week'] = export_df['created_at'].dt.day_name()
    export_df['is_weekend'] = export_df['created_at'].dt.weekday >= 5
    export_df['engagement_rate'] = (
        export_df['total_engagement'] / export_df['text_length']
    ).fillna(0)
    
    # Add political content indicators (basic keyword matching)
    political_keywords = [
        'election', 'vote', 'campaign', 'politics', 'government',
        'policy', 'democracy', 'candidate', 'ballot', 'congress'
    ]
    
    export_df['contains_political_terms'] = export_df['text'].str.lower().str.contains(
        '|'.join(political_keywords), na=False
    )
    
    export_df['political_term_count'] = export_df['text'].str.lower().str.count(
        '|'.join(political_keywords)
    )
    
    # Save to CSV
    export_df.to_csv('bluesky_political_data.csv', index=False)
    
    print("💾 Data Export Complete!")
    print("📊 Variables created for analysis:")
    print("   • Temporal variables: hour_posted, day_of_week, is_weekend")
    print("   • Engagement metrics: total_engagement, engagement_rate")
    print("   • Content analysis: contains_political_terms, political_term_count")
    print("   • Basic metrics: text_length, like_count, repost_count, reply_count")
    print("\n📁 Data saved as: bluesky_political_data.csv")
    print("🔄 Ready for import into R, SPSS, Stata, or other statistical software")
    
    # Display summary statistics
    print("\n📈 Summary Statistics for Export:")
    display(export_df.describe())
else:
    print("⚠️ No data to export")
```

### Step 13: Creating a Research Codebook

```python
# Create a codebook for your data
codebook = {
    'Variable Name': [
        'uri', 'text', 'created_at', 'author_did', 'reply_count',
        'repost_count', 'like_count', 'date', 'text_length',
        'total_engagement', 'hour_posted', 'day_of_week',
        'is_weekend', 'engagement_rate', 'contains_political_terms',
        'political_term_count'
    ],
    'Description': [
        'Unique identifier for the post',
        'Full text content of the post',
        'Timestamp when post was created',
        'Unique identifier for the author',
        'Number of replies to the post',
        'Number of times post was reposted',
        'Number of likes on the post',
        'Date of post (without time)',
        'Length of post text in characters',
        'Sum of all engagement (likes + reposts + replies)',
        'Hour of day when post was made (0-23)',
        'Day of week when post was made',
        'Whether post was made on weekend (True/False)',
        'Engagement per character of text',
        'Whether post contains political keywords (True/False)',
        'Count of political keywords in post'
    ],
    'Type': [
        'Text', 'Text', 'DateTime', 'Text', 'Numeric',
        'Numeric', 'Numeric', 'Date', 'Numeric',
        'Numeric', 'Numeric', 'Categorical',
        'Boolean', 'Numeric', 'Boolean',
        'Numeric'
    ]
}

codebook_df = pd.DataFrame(codebook)
codebook_df.to_csv('bluesky_data_codebook.csv', index=False)

print("📚 Research Codebook Created!")
print("📁 Saved as: bluesky_data_codebook.csv")
print("\n📋 Variable Documentation:")
display(codebook_df)
```

---

## Part 7: Research Ethics and Best Practices

### Important Considerations for Political Science Research 🔍

```python
# Display ethical guidelines
ethics_guidelines = """
🔒 RESEARCH ETHICS CHECKLIST FOR SOCIAL MEDIA DATA

✅ IRB Approval
   • Check if your research requires IRB approval
   • Many institutions require approval for social media research
   • Public data may still require ethical review

✅ Data Privacy
   • Use only publicly available data
   • Remove or anonymize personal identifiers when possible
   • Be careful with sensitive political information

✅ API Compliance
   • Respect rate limits (don't make too many requests)
   • Follow Bluesky's Terms of Service
   • Use app passwords, not main account passwords

✅ Research Transparency
   • Document your data collection methods
   • Report any limitations or biases in your data
   • Make your code available for replication

✅ Data Storage
   • Store data securely on approved university systems
   • Don't keep data longer than necessary for research
   • Follow your institution's data retention policies

⚖️ LEGAL CONSIDERATIONS
   • Public posts are generally okay to analyze
   • Be aware of changing platform policies
   • Consider international data protection laws (GDPR, etc.)
"""

print(ethics_guidelines)
```

### Step 14: Creating a Methods Section Template

```python
methods_template = """
📝 METHODS SECTION TEMPLATE FOR YOUR RESEARCH PAPER

Data Collection:
Social media data was collected from Bluesky using the AT Protocol API.
Data collection occurred between [START_DATE] and [END_DATE].
We collected [N] posts from [N] political accounts identified through [SAMPLING_METHOD].

Sample:
Our sample included [DESCRIPTION OF ACCOUNTS/USERS].
Accounts were selected based on [SELECTION_CRITERIA].
[LIMITATIONS OF SAMPLE].

Variables:
We analyzed several dimensions of political communication:
• Temporal patterns: posting time, day of week
• Engagement: likes, reposts, replies
• Content: text length, political keywords
• Network: follow relationships between accounts

Analysis:
Data analysis was conducted using Python with the pandas library for data manipulation
and [STATISTICAL_SOFTWARE] for inferential statistics.
[SPECIFIC ANALYTICAL TECHNIQUES USED].

Limitations:
• Data represents only Bluesky users (selection bias)
• Public posts only (may miss private political discussion)
• Limited to English-language content
• [OTHER STUDY-SPECIFIC LIMITATIONS]
"""

print("📄 Methods Section Template:")
print(methods_template)

# Save template to file
with open('methods_template.txt', 'w') as f:
    f.write(methods_template)

print("\n💾 Template saved as: methods_template.txt")
```

---

## Part 8: Advanced Research Applications

### Step 15: Longitudinal Analysis Framework

```python
def setup_longitudinal_study():
    """
    Framework for collecting data over time
    Useful for studying political campaigns, crises, etc.
    """
    framework = """
    🗓️ LONGITUDINAL STUDY FRAMEWORK
    
    Pre-Event Phase (Baseline):
    • Collect data 2-4 weeks before key political event
    • Establish normal patterns of discourse
    • Identify key actors and themes
    
    Event Phase:
    • Increase data collection frequency during event
    • Monitor real-time reactions and discussions
    • Track information spread and engagement spikes
    
    Post-Event Phase:
    • Continue collection for 2-4 weeks after event
    • Analyze sustained vs. temporary changes
    • Document new network formations
    
    Research Questions for Longitudinal Analysis:
    • How does political discourse change during crises?
    • What information spreads fastest in political networks?
    • How do political communities form and dissolve?
    • What factors predict viral political content?
    """
    return framework

print(setup_longitudinal_study())
```

### Step 16: Comparative Analysis Setup

```python
def comparative_analysis_guide():
    """
    Guide for comparing different political actors or time periods
    """
    guide = """
    🔍 COMPARATIVE ANALYSIS FRAMEWORK
    
    Cross-Actor Comparison:
    • Compare politicians vs. journalists vs. activists
    • Analyze different political parties or ideologies
    • Study institutional vs. individual accounts
    
    Cross-Platform Comparison:
    • Compare Bluesky discourse to Twitter/X
    • Analyze migration patterns between platforms
    • Study platform-specific political behaviors
    
    Cross-Time Comparison:
    • Compare pre/post election periods
    • Analyze seasonal patterns in political engagement
    • Study long-term discourse evolution
    
    Key Metrics for Comparison:
    • Engagement rates and patterns
    • Network centrality and influence
    • Content themes and sentiment
    • Information sharing behaviors
    """
    return guide

print(comparative_analysis_guide())
```

---

## Part 9: Troubleshooting and Next Steps

### Common Issues and Solutions 🛠️

```python
troubleshooting_guide = """
❓ COMMON ISSUES AND SOLUTIONS

Issue: "No posts collected"
Solutions:
• Check if accounts are active and public
• Verify account handles are correct
• Try different political accounts
• Check your internet connection

Issue: "Rate limit exceeded"
Solutions:
• Add time.sleep(1) between API calls
• Reduce the number of requests
• Spread data collection over longer periods

Issue: "Empty dataframes"
Solutions:
• Verify accounts have recent posts
• Check date filters aren't too restrictive
• Try accounts with more activity

Issue: "Authentication errors"
Solutions:
• Use app passwords, not main passwords
• Check Bluesky account settings
• Verify credentials are correct

Issue: "Missing data in analysis"
Solutions:
• Check for None/null values in data
• Verify data types are correct
• Use .fillna() for missing values
"""

print(troubleshooting_guide)
```

### Next Steps for Advanced Research 🚀

```python
next_steps = """
🎯 NEXT STEPS FOR ADVANCED POLITICAL RESEARCH

Immediate Next Steps:
1. Expand your sample size with more political accounts
2. Implement automated data collection for longitudinal studies
3. Add sentiment analysis to your content analysis
4. Create more sophisticated network visualizations

Intermediate Developments:
• Learn natural language processing (NLP) for content analysis
• Implement machine learning for political topic modeling
• Develop automated political event detection
• Create real-time political monitoring dashboards

Advanced Research Directions:
• Study political information cascades and viral spread
• Analyze political polarization through network analysis
• Develop predictive models for political engagement
• Create comparative studies across multiple platforms

Additional Resources:
• Political Science + Data Science: "Bit by Bit" by Matthew Salganik
• Network Analysis: "Networks, Crowds, and Markets" by Easley & Kleinberg
• Text Analysis: "Text Analysis with R" by Silge & Robinson
• Social Media Research: "Digital Sociology" by Lupton

🔗 Useful Libraries for Future Development:
• TextBlob or VADER for sentiment analysis
• scikit-learn for machine learning
• Plotly for interactive visualizations
• Streamlit for creating web applications
"""

print(next_steps)
```

---

## Part 10: Assignment Ideas for Instructors 👩‍🏫

```python
assignment_ideas = """
📚 ASSIGNMENT IDEAS FOR POLITICAL SCIENCE COURSES

Beginner Assignments (Week 1-2):
1. Data Collection Portfolio
   • Collect 100 posts from 5 political accounts
   • Create basic visualizations of posting patterns
   • Write 500-word methodology reflection

2. Political Network Mapping
   • Map follow relationships among 10 political accounts
   • Identify central and peripheral actors
   • Discuss implications for political influence

Intermediate Assignments (Week 3-4):
3. Campaign Communication Analysis
   • Compare pre/post election communication patterns
   • Analyze engagement differences across account types
   • Create presentation of findings

4. Crisis Communication Study
   • Analyze political discourse during major news event
   • Track information spread and reaction patterns
   • Write research report with policy implications

Advanced Projects (Semester-long):
5. Longitudinal Political Engagement Study
   • Track political accounts over full semester
   • Analyze seasonal patterns and event responses
   • Create comprehensive research paper

6. Comparative Platform Analysis
   • Compare political discourse on Bluesky vs. other platforms
   • Develop theoretical framework for platform differences
   • Present findings at student research conference

Assessment Criteria:
• Data collection methodology (25%)
• Analysis sophistication (25%)
• Visualization quality (20%)
• Written communication (20%)
• Code documentation (10%)
"""

print(assignment_ideas)
```

---

## Conclusion 🎓

**Congratulations!** You've completed the Bluesky API tutorial for political science research. You now have the tools to:

- ✅ Collect social media data using the AT Protocol
- ✅ Analyze political discourse patterns
- ✅ Visualize political networks and engagement
- ✅ Export data for statistical analysis
- ✅ Conduct ethical social media research

### Key Takeaways for Political Scientists:

1. **Social media data provides unprecedented insights** into political behavior and discourse
2. **Network analysis reveals power structures** and information flow patterns
3. **Temporal analysis helps understand** how political events shape discourse
4. **Ethical considerations are paramount** in social media research
5. **Replication and transparency** are essential for credible research

### Final Tips:
- Start small and build complexity gradually
- Always document your methods thoroughly
- Collaborate with computer science colleagues when possible
- Stay updated on platform changes and new research methods
- Remember that code is a tool for answering political questions, not an end in itself

**Happy researching!** 🗳️📊🔬

---

*This tutorial was created for educational purposes. Always follow your institution's IRB guidelines and respect platform terms of service when conducting research.*